# Frozenlake Value Iteration exercise
Below you can find an implementation for solving the Frozenlake problem using the Value Iteration algorithm. 
 
**Tasks:**
1. After running the code to verify that the agent can solve the problem, you can run the code below to visualize the agent playing the game, using the optimal policy found by the Value Iteration algorithm.
2. Make a plot with the evolution of the reward obtained by the agent during the training process.
3. Create a visualization of the optimal policy found by the agent. You can do it similarly to what was done for the gridworld problem in the previous chapter.

In [2]:
import gymnasium as gym
import collections
from tensorboardX import SummaryWriter

In [3]:
ENV_NAME = "FrozenLake-v1"
GAMMA = 0.9
TEST_EPISODES = 20

In [4]:
class Agent:
    def __init__(self, render=None):
        self.env = gym.make(ENV_NAME, render_mode=render)
        self.state, info = self.env.reset()
        self.values = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.rewards = collections.defaultdict(float)
    
    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _, info = self.env.step(action)
            self.transits[self.state, action][new_state] += 1
            self.rewards[self.state, action, new_state] = reward
            self.state, info = self.env.reset() if is_done else (new_state, {})
    
    def calc_action_value(self, state, action):
        target_counts = self.transits[state, action]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[state, action, tgt_state]
            val = reward + GAMMA * self.values[tgt_state]
            action_value += (count / total) * val
        return action_value
    
    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or (best_value < action_value):
                best_value = action_value
                best_action = action
        return best_action
    
    def play_episode(self, env):
        total_reward = 0.0
        state, info = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _, info = env.step(action)
            self.transits[state, action][new_state] += 1
            self.rewards[(state, action, new_state)] = reward
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward
    
    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [self.calc_action_value(state, action) for action in range(self.env.action_space.n)]
            self.values[state] = max(state_values)
            

In [6]:
test_env = gym.make(ENV_NAME, render_mode="human")
test_env.metadata['render_fps']=0 # Velocidade máxima de renderização
agent = Agent()
writer = SummaryWriter(comment="-v-iteration")
iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    agent.play_n_random_steps(100)
    agent.value_iteration()
    
    reward = 0.0
    for _ in range(TEST_EPISODES):    
        reward += agent.play_episode(test_env)
        print(f"Iteration {iter_no} - Test run {_}, reward: {reward}\r", end="" if _< TEST_EPISODES-1 else "\n")
    reward /= TEST_EPISODES
    writer.add_scalar("reward", reward, iter_no)
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
        best_reward = reward
    if reward > 0.80:
        print(f"Solved in {iter_no} iterations!")
        break
writer.close()

/mnt/Datos/MEGAsync/Cursos/Reinforcement_Learning/Reinforcement-Learning-course/.venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Iteration 1 - Test run 19, reward: 0.0
Iteration 2 - Test run 19, reward: 0.0
Iteration 3 - Test run 19, reward: 0.0
Iteration 4 - Test run 19, reward: 0.0
Iteration 5 - Test run 19, reward: 0.0
Iteration 6 - Test run 19, reward: 0.0
Iteration 7 - Test run 19, reward: 0.0
Iteration 8 - Test run 19, reward: 0.0
Iteration 9 - Test run 19, reward: 0.0
Iteration 10 - Test run 19, reward: 0.0
Iteration 11 - Test run 19, reward: 0.0
Iteration 12 - Test run 19, reward: 0.0
Iteration 13 - Test run 19, reward: 0.0
Iteration 14 - Test run 19, reward: 0.0
Iteration 15 - Test run 19, reward: 0.0
Iteration 16 - Test run 19, reward: 0.0
Iteration 17 - Test run 19, reward: 0.0
Iteration 18 - Test run 19, reward: 0.0
Iteration 19 - Test run 19, reward: 3.0
Best reward updated 0.000 -> 0.150
Iteration 20 - Test run 19, reward: 0.0
Iteration 21 - Test run 19, reward: 7.0
Best reward updated 0.150 -> 0.350
Iteration 22 - Test run 19, reward: 12.0
Best reward updated 0.350 -> 0.600
Iteration 23 - Test run